<a href="https://colab.research.google.com/github/joshmcadams/nanoGPT/blob/main/nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nanoGPT

nanoGPT is a wonderful exploration into building a GPT model from scratch. We'll start with an empty notebook and quickly build and train a model to generate Shakespeare-like text.

This notebook is based off of a code and videos releasted by Andrej Karpathy. Please be sure to check out his work!

  * [Reference Video Lecture](https://www.youtube.com/watch?v=kCc8FmEb1nY)
  * [Reference Git Repository](https://github.com/karpathy/ng-video-lecture)

# Data Exploration and Preparation

## Acquiring the data

We'll first pull the data down into this lab. To get the data the `wget` command will be used in the shell.

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-23 23:28:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-03-23 23:28:13 (17.6 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



Next we'll load the data into Python and take a look at a sample.

First, we simply open the file and store the entire contents of the file in the variable `raw_training_data`. We get the length of the data and print that out so that we can see how much data we are dealing with.

In [17]:
with open('input.txt', 'r', encoding='utf-8') as f:
    raw_training_data = f.read()

training_data_size = len(raw_training_data)
print(training_data_size)

Next we sample the data just to get an idea of what we are dealing with. The sample comes from a random locaiton in the data. Run the code block below a few times to see some different data samples.

In [21]:
import random

SAMPLE_SIZE = 500
start = random.randrange(0, training_data_size - SAMPLE_SIZE)
print(raw_training_data[start:start+SAMPLE_SIZE])

.

PAULINA:
I pray now, call her.
Withdraw yourselves.

Gaoler:
And, madam,
I must be present at your conference.

PAULINA:
Well, be't so, prithee.
Here's such ado to make no stain a stain
As passes colouring.
Dear gentlewoman,
How fares our gracious lady?

EMILIA:
As well as one so great and so forlorn
May hold together: on her frights and griefs,
Which never tender lady hath born greater,
She is something before her time deliver'd.

PAULINA:
A boy?

EMILIA:
A daughter, and a goodly babe,
Lusty


## Data Analysis

We've now acquired the data and even poked around a bit to see what the data looks like. However, we need to dig deeper and do some more detailed analysis on the data.

One important fact to know about data that will be fed into a generative model is how many different characters are we working with. This is actually really easy in Python and can be accomplished using the `set` function.

In [28]:
tokens = sorted(list(set(raw_training_data)))
token_count = len(tokens)

print(f'There are {token_count} unique characters in the {training_data_size}' +
      f'of training data. The characters are: {"".join(tokens)}')

There are 65 unique characters in the 1115394of training data. The characters are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


## Encoding

We now know how many different unique tokens exist in our training data. In our case, these tokens will be individual characters, but this a choice that we have made. We could have alternatively chosen pairs, triplets, or some other combination of characters to be a token.

Regardless of the data that comprises our tokens, the token itself isn't what we feed to the model. Instead, we need to convert the token into a numeric representation since models are performing numeric calculations internally.

For our character-based tokens we could just use the ASCII value of the character using the `ord` function. However, you might eventually want to expermiment with sequences of characters as tokens, and in this case you'll need a more complex token-to-number mapping, so lets just create a token-to-number mapping scheme now.

Since we ordered our unique tokens into the `tokens` list, we can just use the index of the token in the list as the encoding, which is what we do.

Note that this does come with some trade-offs though. The position of any given token in the list of tokens is relative to how many and what were the unique tokens in the training data. If we use different trianing data, the tokens might map to different indexes so we need to be careful to preserve our tokens across training data set if we are using mulitple different pieces of training data.

Anyway, let's write a token-to-number `encode` function and a number-to-token `decode` function.

First, create a mapping of tokens to numbers.

In [34]:
token_to_number = {t: i for i, t in enumerate(tokens)}
token_to_number

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

And then a mapping from numbers to tokens.

In [35]:
number_to_token = {i: t for i, t in enumerate(tokens)}
number_to_token

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

And now we can write our encoder.

In [42]:
def encode(tokens: str) -> list[int]:
  return [token_to_number[t] for t in tokens]

encode('apple')

[39, 54, 54, 50, 43]

And the decoder.

In [45]:
def decode(numbers: list[int]) -> str:
  return ''.join(number_to_token[n] for n in numbers)

decode([16, 17, 18])

'DEF'